In [1]:
'''
VMP 2022-02-22: 
key document for preprocessing main scientific fields. 
doing some edits and sanity checking.
'''

'\nVMP 2022-02-22: \nkey document for preprocessing main scientific fields. \ndoing some edits and sanity checking.\n'

In [2]:
import sys  
sys.path.insert(0, '/home/vicp/reform-psychology/MAG-data-curation')
from MAGsparkmasters import get_mag_with_cluster_connection
from MAGmasters import MicrosoftAcademicGraph
import os
from pyspark.sql import functions as F, Window
import pandas as pd
import matplotlib.pyplot as plt
import datetime, time
import numpy as np
from pyspark.sql.functions import avg
os.chdir('/home/vicp/reform-psychology/MAG-data-curation')
mag, spark = get_mag_with_cluster_connection(67190, 
                               memory_per_executor=16000)

['NAME STATE JOBID', 'cluster_new.job RUNNING 67190', 'cluster_new.job RUNNING 67189', 'main_job RUNNING 67150', 'AHC RUNNING 67090', 'jupyter-notebook RUNNING 67185', 'run_all_benchmarks.job RUNNING 67120', 'countrydata RUNNING 67116', '1.train.1 RUNNING 67164', '1.train.2 RUNNING 67165', '1.train.7 RUNNING 67170', '']


In [3]:
spark

# NormalizedName
Create file called *FoS.txt*: <br/>
File contains *COLUMNS*: 
* FieldOfStudyId <int>
* NormalizedName <str>
* Level <int>

In [4]:
def NormalizedName(): 
    
    # NormalizedName and FieldOfStudyId mapping. 
    FoS = mag.getSubset("FieldsOfStudy", ['FieldOfStudyId', 'NormalizedName', 'Level']) \
        .distinct() 
    print(f"dtypes: {FoS.dtypes}")
    
    # write file 
    mag.saveFile(FoS, "masters", "FoS.txt")

In [5]:
NormalizedName()

dtypes: [('FieldOfStudyId', 'bigint'), ('NormalizedName', 'string'), ('Level', 'int')]


# Subset by FoS
based on a list of fields-of-study, we create subsets.

In [6]:
focus_list = ['sociology', 'political science', 'psychology', 'economics']

In [7]:
focus_fos = mag.getDataframe('FoS') \
    .filter(F.col('NormalizedName').isin(focus_list)) \
    .toPandas()

In [8]:
focus_fos.head()

,FieldOfStudyId,NormalizedName,Level
0,162324750,economics,0
1,17744445,political science,0
2,144024400,sociology,0
3,15744967,psychology,0


In [9]:
## one way of doing it ##
# could just inner join & take NormalizedName instead. 
# actually, at this point we only need PaperId. 

In [10]:
# create dataframes 
def fos_papers(fos, name): 
    # quick preprocessing
    pfs = mag.getDataframe('PaperFieldsOfStudy') \
        .filter(F.col('FieldOfStudyId').isin(fos)) \
        .select('PaperId') \
        .distinct() 
    print(f"dtypes {name}: {pfs.dtypes}")
    # write file 
    mag.saveFile(pfs, "masters", f"{name}_papers.txt")

In [11]:
for index, row in focus_fos.iterrows():
    fos_papers(row['FieldOfStudyId'], row['NormalizedName'].replace(" ", ""))

dtypes economics: [('PaperId', 'bigint')]
dtypes politicalscience: [('PaperId', 'bigint')]
dtypes sociology: [('PaperId', 'bigint')]
dtypes psychology: [('PaperId', 'bigint')]


# Subset by Year (2005-2021) and DocType (Journal)
including 2010 and and 2020. (not including 2021). <br/>
only doctype journal. <br/>
subset needed to filter PaperReferences. 
Here, again - we do not need FieldOfStudyId. 

In [12]:
### this works, but two thigns 
# (1) we do not need FieldOfStuyId - and
# (2) we need to write this as a .csv as well

In [15]:
def subset_data(fos): 
    papers = mag.getDataframe('Papers') # PaperId, DocType, Date, FamilyId. 
    fos_papers = mag.getDataframe(f'{fos}_papers') # FieldOfStudyID, PaperId
    
    # filter papers first - core data set. 
    sub = papers.filter((F.col('DocType') == 'Journal') | (F.col('DocType') == 'Conference')) \
        .filter((F.col('Date') >= datetime.date(2005, 1, 1)) & (F.col('Date') <= datetime.date(2021, 1, 1))) \
        .join(fos_papers, ['PaperId'], 'inner') \
        .select('PaperId', 'FamilyId') \
        .distinct()
    print(f"{fos} dtypes: {sub.dtypes}")
    
    mag.saveFile(sub, "masters", f"{fos}_subset.txt")
    sub.toPandas().to_csv(f"/home/vicp/data/2021-08-02/masters/{fos}_subset.csv", index=False)

In [16]:
focus_list = ['sociology', 'politicalscience', 'psychology', 'economics']
for fos in focus_list:
    subset_data(fos)

sociology dtypes: [('PaperId', 'bigint'), ('FamilyId', 'bigint')]
politicalscience dtypes: [('PaperId', 'bigint'), ('FamilyId', 'bigint')]
psychology dtypes: [('PaperId', 'bigint'), ('FamilyId', 'bigint')]
economics dtypes: [('PaperId', 'bigint'), ('FamilyId', 'bigint')]


In [17]:
## write to csv ## 
# really should delete FieldOfStudyId

# PaperReferences (v2)

In [18]:
# this works 

In [21]:
def get_citation_reference(fos):
    '''
    ## input
    fos: field of study (without space)
    
    ## output / save
    {fos}_reference_or_citation.txt: <folder> one column of PaperId that either is reference or citation of fos. 
    {fos}_citation: <csv> the subset of PaperReferences (two columns) -- where the fos cites other articles. 
    {fos}_reference: <csv> the subset of PaperReferences (two columns) -- where the fos are references by other articles.
    '''
    
    # load files
    pr = mag.getDataframe('PaperReferences')
    fos_subset = mag.getDataframe(f'{fos}_subset').select('PaperId')
    
    ## the papers that cite our focus papers
    ## i.e. our papers have to be PaperReferenceId (no?) --> should be LEFT
    reference = fos_subset.withColumnRenamed('PaperId', 'PaperReferenceId') \
        .join(pr, ['PaperReferenceId'], "inner") \
        .distinct()
    

    ## this should give us all papers (we will gather meta-data for this): 
    # (1) FoS: from particular field of study (from fos_subset)
    # (2) those that have selected field of study as reference (from reference)
    # need to combine both to get (1) references, but also (2) psychology papers with no citations. 
    papers_total = reference.select('PaperId').join(fos_subset, 'PaperId', 'outer').distinct()

    # save stuff
    print(f"{fos} papers_total: {papers_total.dtypes}")
    mag.saveFile(papers_total, "masters", f"{fos}_total.txt")
    reference.toPandas().to_csv(f"/home/vicp/data/2021-08-02/masters/{fos}_reference.csv", index = False)

In [ ]:
focus_list = ['psychology', 'sociology', 'politicalscience', 'economics']
for fos in focus_list:
    get_citation_reference(fos)

psychology papers_total: [('PaperId', 'bigint')]
sociology papers_total: [('PaperId', 'bigint')]


# Add Field Of Study to papers we care about

# metadata
we add metadata to the subsetted fields. <br/>
we should enforce no na in PaperId & Date (at least) <br/>
NB: we loose a bit of data in .filter(F.col('Level') == 0) as not all studies have a level 0. <br/>
NB: some studies also do not have any match in PaperFieldOfStudy. <br/>
NB: I think generally studies that were hard to classify and are fine to leave out (filtered on Ucloud). <br/>
I filter out some papers based on Max Score (i.e. papers with more than one top-level field of study I take the one that MAG is most confident of - e.g. Psychology: 0.73, Biology: 0.51 I take Psychology): <br/>
https://stackoverflow.com/questions/48829993/groupby-column-and-filter-rows-with-maximum-value-in-pyspark

In [ ]:
def get_metadata_papers(fos):
    
    # load files 
    papers = mag.getSubset('Papers', ['PaperId', 'PaperTitle', 'DocType', 'Date', 'FamilyId']) 
    fos_papers = mag.getDataframe(f'{fos}_total') # PaperId
    pfs = mag.getDataframe('PaperFieldsOfStudy') # couples PaperId and FieldOfStudyId
    FoS = mag.getDataframe('FoS') # couples FieldOfStudyId and NormalizedName
    
    # window 
    w = Window.partitionBy('PaperId')
    
    # filter papers first - core data set. 
    fos_papers.join(papers, ['PaperId'], 'inner') \
        .join(pfs, ['PaperId'], 'inner') \
        .join(FoS, ['FieldOfStudyId'], 'inner') \
        .filter(F.col('Level') == 0) \
        .withColumn('MaxScore', F.max('Score').over(w)) \
        .where(F.col('Score') == F.col('MaxScore')) \
        .select('PaperId', 'FamilyId', 'PaperTitle', 'Date', 'DocType', 'NormalizedName') \
        .distinct() \
        .toPandas() \
        .to_csv(f"/home/vicp/data/2021-08-02/masters/{fos}_paper_meta_clean.csv", index = False)
    print(f"-- finished processing {fos}")
    

In [ ]:
focus_list = ['psychology', 'sociology', 'politicalscience', 'economics']
for fos in focus_list: 
    get_metadata_papers(fos)

In [ ]:
# in the below we could consider only adding meta-data to the records in "meta". 

In [ ]:
def get_paper_author(fos): 
    # load relevant data
    fos_papers = mag.getDataframe(f'{fos}_total') # PaperId
    author_affiliations = mag.getSubset('PaperAuthorAffiliations', ['PaperId', 'AuthorId']) # PaperId, AuthorId. 
    
    # inner join
    fos_papers.join(author_affiliations, ['PaperId'], "inner") \
        .distinct() \
        .toPandas() \
        .to_csv(f"/home/vicp/data/2021-08-02/masters/{fos}_paper_author.csv", index = False)
    print(f"-- finished processing {fos}")

In [ ]:
focus_list = ['psychology', 'sociology', 'politicalscience', 'economics']
for fos in focus_list: 
    get_paper_author(fos)